In [1]:
from math import sqrt, exp
import pydotplus as pydot
import numpy as np

In [18]:
class cOpt(object):
    def __init__(self, S0, K, rate, sigma, n_periods, tyears,cp):
        self.S0 = S0 # Today's stock price
        self.K = K # Strike Price
        self.rate = rate
        self.sigma = sigma
        self.tyears = tyears
        self.n_periods = n_periods
        self.cp=cp
        #Calcs
        self.up_factor = exp(self.sigma * sqrt(self.tyears / self.n_periods))
        self.down_factor = 1 / self.up_factor
        self.tree = self.option_tree()
        self.option_price = self.tree["option_tree"][0][0]
        
        
    def option_tree(self):
        p = (np.exp(self.rate * self.tyears / self.n_periods) - self.down_factor) / (self.up_factor - self.down_factor)

        # Option Payoff
        contingent_payoff = lambda st, k,cp: max((st - k)*cp, 0)
        contingent_payoff = np.vectorize(contingent_payoff)

        # Creating the branches at different times of the tree. Each branch is the possible martingale price
        # of the stock at time t
        stock_price_tree = []
        for i in range(1,self.n_periods+1):
            # Number of prices per period
            period_prices = []
            for j in range(i+1):
                up_factor_repeat = i-j
                down_factor_repeat = j

                values = np.append(np.repeat(self.up_factor, up_factor_repeat), 
                        np.repeat(self.down_factor, down_factor_repeat))
                stock_price = values.prod() * self.S0
                period_prices.append(stock_price)
            stock_price_tree.append(period_prices)

        # Getting the payoff of the call at the end of the tree
        payoffs = contingent_payoff(stock_price_tree[self.n_periods-1], self.K,self.cp).tolist()

        option_prices = [payoffs]
        # Contingent option price at period n-1
        branch_n = payoffs # brach at n-1

        for t in range(self.n_periods):
            branch_nm1 = []
            for i in range(len(branch_n)-1):
                #cu: Contingent price at time n if price rises
                #cd: Contingent price at time n if price decreases
                cu, cd = np.array(branch_n)[[i, i+1]]
                price = np.exp(-self.rate * self.tyears / self.n_periods)*(p*cu + (1-p)*cd)
                branch_nm1.append(price)
            option_prices.append(branch_nm1)
            branch_n = branch_nm1

        option_prices = option_prices[::-1] 
        stock_price_tree.insert(0, [self.S0])

        return {"stock_tree": stock_price_tree,
                "option_tree": option_prices}
    
        
    def graph_tree(self, file_name = ""):
        option_prices = self.tree["option_tree"]
        stock_price_tree = self.tree["stock_tree"]

        graph = pydot.Dot(graph_type='digraph', rankdir='LR')

        branch = 0
        counter = 1
        for i, prices in enumerate(option_prices[:-1]):
            next_prices = option_prices[i+1]
            for np, c_price in enumerate(prices):
                c_price = str(round(c_price,2))

                nprice1 = str(round(next_prices[np], 2))
                nprice2 = str(round(next_prices[np+1], 2))

                increment = len(prices)-1
                from_branch, to_branch1, to_branch2 = branch, counter + increment, counter + increment + 1
                #print(from_branch,"->",to_branch1, to_branch2)

                up_price = round(stock_price_tree[i][np] * self.up_factor, 2)
                down_price = round(stock_price_tree[i][np] * self.down_factor, 2)
                edge = pydot.Edge(from_branch, to_branch1, label = str(up_price)); graph.add_edge(edge)
                edge = pydot.Edge(from_branch, to_branch2, label = str(down_price)); graph.add_edge(edge)

                node = pydot.Node(to_branch1, label = str(nprice1)); graph.add_node(node)
                node = pydot.Node(to_branch2, label = str(nprice2)); graph.add_node(node)

                counter += 1
                branch += 1
        
        c0 = round(self.option_price, 2)
        node = pydot.Node("0", label = str(c0)); graph.add_node(node)
        graph.write_png(file_name)

In [26]:
option=cOpt(100,100,0.03,.30,100,1,1)

In [27]:
print(option.option_price)

13.253838123630688


In [28]:
%timeit cOpt(100,100,0.03,.30,100,1,1)

1 loop, best of 3: 418 ms per loop


In [13]:
print(option.tree)

In [9]:
option.graph_tree('option_tree.png')